In [2]:
import transformers
from huggingface_hub import interpreter_login
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset


interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [3]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", torch_dtype="auto", device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [4]:
tokens = tokenizer("Roses are red and honey", return_tensors='pt').to('mps')
tokens

{'input_ids': tensor([[128000,     49,  20274,    527,   2579,    323,  26828]],
       device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='mps:0')}

In [5]:
response_tokens = model.generate(**tokens, max_length=30)
response_tokens

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000,     49,  20274,    527,   2579,    323,  26828,    374,  10437,
             13,   1442,    499,    527,   3411,    369,    264,  10437,   4322,
            430,    374,    439,   4382,    439,    433,   5334,     11,   1243,
            499,    690,   1390]], device='mps:0')

In [6]:
tokenizer.batch_decode(response_tokens)

['<|begin_of_text|>Roses are red and honey is sweet. If you are looking for a sweet treat that is as simple as it gets, then you will want']

In [ ]:
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = load_dataset("text", data_files={"train": "agent.txt"})
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Define training arguments.
training_args = TrainingArguments(
    output_dir="./llama-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,                   # Adjust the number of epochs as needed
    per_device_train_batch_size=4,        # Adjust based on your GPU memory
    save_steps=500,                       # Frequency (in steps) to save checkpoints
    save_total_limit=2,                   # Maximum number of checkpoints to keep
    logging_steps=100,                    # Log training info every 100 steps
    # evaluation_strategy="no",             # No evaluation; add one if you have a validation set
)

# Create the Trainer instance for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

# Start the fine-tuning process
trainer.train()

# Save the fine-tuned model and tokenizer for later use
trainer.save_model("./llama-finetuned")
tokenizer.save_pretrained("./llama-finetuned")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/11774 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid de

Step,Training Loss
100,1.910400
200,1.228100
